-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

## Intro to Neural Networks with Keras II

Congrats on building your first neural network! In this notebook, we will cover even more topics to improve your model building. After you learn the concepts here, you will apply them to the neural network you just created.

We will use the California Housing Dataset.

Objectives:
   * Data Normalization
   * Custom Metrics
   * Validation data
   * Checkpointing/callbacks
   * Saving Models

In [3]:
from sklearn.datasets.california_housing import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(0)

cal_housing = fetch_california_housing()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                        cal_housing.target,
                                                        test_size=0.2,
                                                        random_state=1)

print(cal_housing.DESCR)

California housing dataset.

The original database is available from StatLib

 http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.

Let's take a look at the distribution of our features.

In [5]:
import pandas as pd

xTrainDF = pd.DataFrame(X_train, columns=cal_housing.feature_names)

#print(xTrainDF.describe())


MedInc HouseAge AveRooms AveBedrms Population \
count 16512.000000 16512.000000 16512.000000 16512.000000 16512.000000 
mean 3.876149 28.604469 5.441114 1.099598 1425.257146 
std 1.891584 12.586046 2.613727 0.507173 1123.756792 
min 0.499900 1.000000 0.846154 0.333333 3.000000 
25% 2.572050 18.000000 4.439906 1.006260 786.000000 
50% 3.544550 29.000000 5.226528 1.048797 1164.000000 
75% 4.750000 37.000000 6.057778 1.099574 1723.000000 
max 15.000100 52.000000 141.909091 34.066667 35682.000000 

 AveOccup Latitude Longitude 
count 16512.000000 16512.000000 16512.000000 
mean 3.094971 35.632194 -119.574288 
std 11.597402 2.137087 2.007578 
min 0.750000 32.540000 -124.300000 
25% 2.427283 33.930000 -121.810000 
50% 2.813449 34.260000 -118.490000 
75% 3.273834 37.710000 -118.010000 
max 1243.333333 41.950000 -114.310000

## 1. Data Normalization

Because our features are all on different scales, it's going to be more difficult for our neural network during training. Let's do feature-wise normalization.

We are going to use the [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from Sklearn, which will remove the mean (zero-mean) and scale to unit variance.

$$x' = \frac{x - \bar{x}}{\sigma}$$

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Keras Model
![Life Cycle](https://brookewenig.github.io/img/DL/Life-Cycle-for-Neural-Network-Models-in-Keras.png)

In [9]:
X_train.shape

Out[ 5 ]: (16512, 8)

In [10]:
import tensorflow as tf
tf.set_random_seed(42) # For reproducibility

from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
  Dense(20, input_dim=8, activation='relu'),
  Dense(20, activation='relu'),
  Dense(1, activation='linear')
])

/databricks/python/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
 from ._conv import register_converters as _register_converters
Using TensorFlow backend.

## 2. Custom Metrics

Up until this point, we used MSE as our loss function and metric of choice. But what if we want to use RMSE?

In [12]:
model.compile(optimizer="adam", loss="rmse")

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-4254278255566291> in <module> () 
 ----> 1 model . compile ( optimizer = "adam" , loss = "rmse" ) 

 /databricks/python/lib/python3.5/site-packages/keras/models.py in compile (self, optimizer, loss, metrics, sample_weight_mode, weighted_metrics, target_tensors, **kwargs) 
 822 weighted_metrics = weighted_metrics , 
 823 target_tensors = target_tensors , 
 --> 824 **kwargs)
 825 self . optimizer = self . model . optimizer 
 826 self . loss = self . model . loss 

 /databricks/python/lib/python3.5/site-packages/keras/engine/training.py in compile (self, optimizer, loss, metrics, loss_weights, sample_weight_mode, weighted_metrics, target_tensors, **kwargs) 
 634 loss_functions = [ losses . get ( l ) for l in loss ] 
 635 else : 
 --> 636 loss_function = losses . get ( loss ) 
 637 loss_functions = [ loss_function for _ in range ( len ( self . outputs ) ) ] 
 638 self . loss_functions = loss_functions 

 /databricks/python/lib/python3.5/site-packages/keras/losses.py in get (identifier) 
 120 if isinstance ( identifier , six . string_types ) : 
 121 identifier = str ( identifier ) 
 --> 122 return deserialize ( identifier ) 
 123 if isinstance ( identifier , dict ) : 
 124 return deserialize ( identifier ) 

 /databricks/python/lib/python3.5/site-packages/keras/losses.py in deserialize (name, custom_objects) 
 112 module_objects = globals ( ) , 
 113 custom_objects = custom_objects , 
 --> 114 printable_module_name='loss function')
 115 
 116 

 /databricks/python/lib/python3.5/site-packages/keras/utils/generic_utils.py in deserialize_keras_object (identifier, module_objects, custom_objects, printable_module_name) 
 162 if fn is None : 
 163 raise ValueError('Unknown ' + printable_module_name +
 --> 164 ':' + function_name)
 165 return fn 
 166 else : 

 ValueError : Unknown loss function:rmse

Looks like we can't use it in our loss function. What about the metrics we print out during the evaluation?

In [14]:
model.compile(optimizer="adam", loss="mse", metrics=["rmse"])

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-4254278255566293> in <module> () 
 ----> 1 model . compile ( optimizer = "adam" , loss = "mse" , metrics = [ "rmse" ] ) 

 /databricks/python/lib/python3.5/site-packages/keras/models.py in compile (self, optimizer, loss, metrics, sample_weight_mode, weighted_metrics, target_tensors, **kwargs) 
 822 weighted_metrics = weighted_metrics , 
 823 target_tensors = target_tensors , 
 --> 824 **kwargs)
 825 self . optimizer = self . model . optimizer 
 826 self . loss = self . model . loss 

 /databricks/python/lib/python3.5/site-packages/keras/engine/training.py in compile (self, optimizer, loss, metrics, loss_weights, sample_weight_mode, weighted_metrics, target_tensors, **kwargs) 
 932 self . metrics_updates += metric_fn . updates 
 933 
 --> 934 handle_metrics ( output_metrics ) 
 935 handle_metrics ( output_weighted_metrics , weights = weights ) 
 936 

 /databricks/python/lib/python3.5/site-packages/keras/engine/training.py in handle_metrics (metrics, weights) 
 899 metric_name = metric_name_prefix + suffix 
 900 else : 
 --> 901 metric_fn = metrics_module . get ( metric ) 
 902 weighted_metric_fn = _weighted_masked_objective ( metric_fn ) 
 903 # Get metric name as string 

 /databricks/python/lib/python3.5/site-packages/keras/metrics.py in get (identifier) 
 73 return deserialize ( config ) 
 74 elif isinstance ( identifier , six . string_types ) : 
 ---> 75 return deserialize ( str ( identifier ) ) 
 76 elif callable ( identifier ) : 
 77 return identifier 

 /databricks/python/lib/python3.5/site-packages/keras/metrics.py in deserialize (config, custom_objects) 
 65 module_objects = globals ( ) , 
 66 custom_objects = custom_objects , 
 ---> 67 printable_module_name='metric function')
 68 
 69 

 /databricks/python/lib/python3.5/site-packages/keras/utils/generic_utils.py in deserialize_keras_object (identifier, module_objects, custom_objects, printable_module_name) 
 162 if fn is None : 
 163 raise ValueError('Unknown ' + printable_module_name +
 --> 164 ':' + function_name)
 165 return fn 
 166 else : 

 ValueError : Unknown metric function:rmse

Luckily, Keras allows you to define custom metrics. So, you might implement RMSE as below.

In [16]:
from keras import backend
 
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [17]:
model.compile(optimizer="adam", loss="mse", metrics=["mse", rmse])

## 3. Validation Data

Let's take a look at the [.fit()](https://keras.io/models/sequential/) method in the docs to see all of the options we have available! 

We can either explicitly specify a validation dataset, or we can specify a fraction of our training data to be used as our validation dataset.

The reason why we need a validation dataeset is to evaluate how well we are performing on unseen data (neural networks will overfit if you train them for too long!).

We can specify `validation_split` to be any value between 0.0 and 1.0 (defaults to 0.0).

In [19]:
history = model.fit(X_train, y_train, validation_split=.2, epochs=10, verbose=2)

Train on 13209 samples, validate on 3303 samples
Epoch 1/10
 - 1s - loss: 1.3549 - mean_squared_error: 1.3549 - rmse: 0.7829 - val_loss: 0.6020 - val_mean_squared_error: 0.6020 - val_rmse: 0.5548
Epoch 2/10
 - 1s - loss: 0.4916 - mean_squared_error: 0.4916 - rmse: 0.5036 - val_loss: 0.4616 - val_mean_squared_error: 0.4616 - val_rmse: 0.4808
Epoch 3/10
 - 1s - loss: 0.4132 - mean_squared_error: 0.4132 - rmse: 0.4637 - val_loss: 0.4240 - val_mean_squared_error: 0.4240 - val_rmse: 0.4647
Epoch 4/10
 - 1s - loss: 0.3944 - mean_squared_error: 0.3944 - rmse: 0.4504 - val_loss: 0.4037 - val_mean_squared_error: 0.4037 - val_rmse: 0.4457
Epoch 5/10
 - 1s - loss: 0.3868 - mean_squared_error: 0.3868 - rmse: 0.4423 - val_loss: 0.3970 - val_mean_squared_error: 0.3970 - val_rmse: 0.4505
Epoch 6/10
 - 1s - loss: 0.4010 - mean_squared_error: 0.4010 - rmse: 0.4368 - val_loss: 0.3840 - val_mean_squared_error: 0.3840 - val_rmse: 0.4379
Epoch 7/10
 - 1s - loss: 0.3671 - mean_squared_error: 0.3671 - rmse: 0.4291 - val_loss: 0.3883 - val_mean_squared_error: 0.3883 - val_rmse: 0.4355
Epoch 8/10
 - 1s - loss: 0.3521 - mean_squared_error: 0.3521 - rmse: 0.4245 - val_loss: 0.3755 - val_mean_squared_error: 0.3755 - val_rmse: 0.4346
Epoch 9/10
 - 1s - loss: 0.3477 - mean_squared_error: 0.3477 - rmse: 0.4202 - val_loss: 0.3685 - val_mean_squared_error: 0.3685 - val_rmse: 0.4359
Epoch 10/10
 - 1s - loss: 0.3458 - mean_squared_error: 0.3458 - rmse: 0.4174 - val_loss: 0.3599 - val_mean_squared_error: 0.3599 - val_rmse: 0.4269

Wow! Look at how much lower our loss is to start, and that it is able to converge more quickly thanks to the data normalization!!

But, let's test: Is that RMSE correct?

In [21]:
import numpy as np

np.sqrt(history.history['mean_squared_error'][-1]) # Get MSE of last training epoch

Out[ 12 ]: 0.588017128065564

#### Gotcha!! 

Because Keras computes the loss batch by batch, if we take the square root of the total MSE, it does not yield the same result as this RMSE function.

You can see Francois Challot's [comment](https://github.com/keras-team/keras/issues/1170) on this issue, recommending to stick with MSE. But for teaching purposes, now you see how to wrtie custom metric functions!

## 4. Checkpointing

After each epoch, we want to save the model. However, we will pass in the flag `save_best_only=True`, which will only save the model if the validation loss decreased. This way, if our machine crashes or we start to overfit, we can always go back to the "good" state of the model.

To accomplish this, we will use the ModelCheckpoint [callback](https://keras.io/callbacks/). History is an example of a callback that is automatically applied to every Keras model.

In [24]:
from keras.callbacks import ModelCheckpoint

filepath = '/tmp/02Keras_checkpoint_weights.hdf5'
checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)

history = model.fit(X_train, y_train, validation_split=.2, epochs=10, verbose=2, callbacks=[checkpointer])

Train on 13209 samples, validate on 3303 samples
Epoch 1/10
 - 1s - loss: 0.3363 - mean_squared_error: 0.3363 - rmse: 0.4133 - val_loss: 0.3552 - val_mean_squared_error: 0.3552 - val_rmse: 0.4163

Epoch 00001: val_loss improved from inf to 0.35518, saving model to /tmp/02Keras_checkpoint_weights.hdf5
Epoch 2/10
 - 1s - loss: 0.3303 - mean_squared_error: 0.3303 - rmse: 0.4083 - val_loss: 0.3579 - val_mean_squared_error: 0.3579 - val_rmse: 0.4301

Epoch 00002: val_loss did not improve
Epoch 3/10
 - 1s - loss: 0.3296 - mean_squared_error: 0.3296 - rmse: 0.4080 - val_loss: 0.3547 - val_mean_squared_error: 0.3547 - val_rmse: 0.4142

Epoch 00003: val_loss improved from 0.35518 to 0.35470, saving model to /tmp/02Keras_checkpoint_weights.hdf5
Epoch 4/10
 - 1s - loss: 0.3256 - mean_squared_error: 0.3256 - rmse: 0.4037 - val_loss: 0.3450 - val_mean_squared_error: 0.3450 - val_rmse: 0.4088

Epoch 00004: val_loss improved from 0.35470 to 0.34496, saving model to /tmp/02Keras_checkpoint_weights.hdf5
Epoch 5/10
 - 1s - loss: 0.3631 - mean_squared_error: 0.3631 - rmse: 0.4050 - val_loss: 0.3455 - val_mean_squared_error: 0.3455 - val_rmse: 0.4113

Epoch 00005: val_loss did not improve
Epoch 6/10
 - 1s - loss: 0.3180 - mean_squared_error: 0.3180 - rmse: 0.3983 - val_loss: 0.3399 - val_mean_squared_error: 0.3399 - val_rmse: 0.4063

Epoch 00006: val_loss improved from 0.34496 to 0.33987, saving model to /tmp/02Keras_checkpoint_weights.hdf5
Epoch 7/10
 - 1s - loss: 0.3179 - mean_squared_error: 0.3179 - rmse: 0.3968 - val_loss: 0.3435 - val_mean_squared_error: 0.3435 - val_rmse: 0.4154

Epoch 00007: val_loss did not improve
Epoch 8/10
 - 1s - loss: 0.3137 - mean_squared_error: 0.3137 - rmse: 0.3946 - val_loss: 0.3374 - val_mean_squared_error: 0.3374 - val_rmse: 0.4113

Epoch 00008: val_loss improved from 0.33987 to 0.33744, saving model to /tmp/02Keras_checkpoint_weights.hdf5
Epoch 9/10
 - 1s - loss: 0.3171 - mean_squared_error: 0.3171 - rmse: 0.3955 - val_loss: 0.3310 - val_mean_squared_error: 0.3310 - val_rmse: 0.4001

Epoch 00009: val_loss improved from 0.33744 to 0.33097, saving model to /tmp/02Keras_checkpoint_weights.hdf5
Epoch 10/10
 - 1s - loss: 0.3083 - mean_squared_error: 0.3083 - rmse: 0.3920 - val_loss: 0.3368 - val_mean_squared_error: 0.3368 - val_rmse: 0.3980

Epoch 00010: val_loss did not improve

## 5. Save Model/Load Model

Whenever you train neural networks, you want to save them. This way, you can reuse them later! With the checkpointing agove, we were saving the model weights. Let's try to load them into a new model.

In [26]:
newModel = Sequential()

newModel.load_weights(filepath)

We just saved our model weights with the checkpointing above. However, we also need the model configuration if we want to load the weights into a new model object.

In [28]:
from keras.models import model_from_yaml

yaml_string = model.to_yaml() # Returns a representation of the model as a YAML string (only model architecture, not weights)
newModel = model_from_yaml(yaml_string)

Check that the model architecture is the same.

In [30]:
newModel.summary()

Now we can load in the weights for this model architecture.

In [32]:
newModel.load_weights(filepath)

Let's train it for one more epoch (we need to recompile), and then save those weights.

In [34]:
newModel.compile(optimizer="adam", loss="mse")
newModel.fit(X_train, y_train, validation_split=.2, epochs=1, verbose=2)
newModel.save_weights(filepath)

Now it's your turn to try out these techniques on the Boston Housing Dataset!

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>